In [35]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [3]:
with open('data/training.csv','r') as f:
    train_data = f.read().split('\n')

In [31]:
train_data = pd.read_csv('data/training.csv',delimiter=";")

In [14]:
train_data['isUS'] = train_data['country_code'] == "US"

In [17]:
train_data['isUS'].replace({False: 0, True: 1}, inplace=True)

In [23]:
train_data_text = list(train_data['text'])
train_data_code = list(train_data['isUS'])

In [60]:
# tokenizing
count_vect = CountVectorizer(stop_words="english")
X_train_counts = count_vect.fit_transform(train_data_text)
X_train_counts.shape

(9999, 38452)

In [61]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(9999, 38452)

In [62]:
# train a classifier
from sklearn.naive_bayes import MultinomialNB

In [63]:
clf = MultinomialNB().fit(X_train_tfidf, train_data_code)

In [64]:
test_sample = train_data_text[2:3]

In [65]:
X_new_counts = count_vect.transform(test_sample)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [66]:
clf.predict(X_new_tfidf)

array([0])

In [67]:
# prepare test data
test_data = pd.read_csv("data/test.csv", delimiter=';')
test_data['isUS'] = test_data['country_code'] == "US"
test_data['isUS'].replace({False: 0, True: 1}, inplace=True)

test_data_text = list(test_data['text'])
test_data_code = list(test_data['isUS'])

# test data tokenizing
count_vect = CountVectorizer(stop_words="english")
X_test_counts = count_vect.fit_transform(test_data_text)
X_test_counts.shape

(29999, 86758)

In [69]:
data_all_text = train_data_text + test_data_text
data_all_code = train_data_code + test_data_code

In [70]:
# tokenizing
count_vect = CountVectorizer(stop_words="english")
X_all_counts = count_vect.fit_transform(data_all_text)
X_all_counts.shape

(39998, 107267)

In [72]:
train_counts = X_all_counts[:9999,:]
test_counts = X_all_counts[9999:,]
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(train_counts)
X_test_tfidf = tfidf_transformer.fit_transform(test_counts)
X_train_tfidf.shape

(9999, 107267)

In [73]:
clf = MultinomialNB().fit(X_train_tfidf, train_data_code)

In [75]:
test_predictions = clf.predict(X_test_tfidf)

In [76]:
np.mean(test_predictions == test_data_code)

0.7664922164072135

In [80]:
# dump model with pickle
import pickle
with open('tweet_classifier.pkl','wb') as f:
    pickle.dump(clf,f)